Text analysis: How does SPON write about men, how about women?
----------
Vast parts of this notebook are copied from the work of [*Neal Caren*](http://twitter.com/HaphazardSoc) - University of North Carolina, Chapel Hill

In [1]:
from __future__ import division

import glob
import nltk
from string import punctuation
import pandas as pd
import re

tokenizer = nltk.data.load('tokenizers/punkt/german.pickle')

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [1]:
#Word lists to identify male/female sentences - this is particularly complicated for German, as words like "sie/Sie, ihre/Ihre" can mean both genders AND are extremly common.
male_words=set(['mann','sprecher','vorsitzender', 'journalist' 'politiker', 'arzt' 'männer', 'mediziner','sein','seine', 'seines', 'ihm', 'er', 'der','partner', 'geliebter','jungen','bruder','brüder','vater','vaters','väter','kollege','verlobter','ehemann','freund','mitschüler','kommilitone','großvater','neffe','bräutigam','enkel','onkel', 'neffe','vorsitzender','präsident','könig','herr','seinem','seiner','priester','prinz','sohn','söhne','fußballer','handballer','sportler','athlet', 'kritiker', 'täter', 'ermittler', 'polizist', 'anwalt', 'staatsanwalt', 'schüler', 'student', 'geschäftsmann', 'spieler', 'asylbewerber', 'bewohner', 'bürgermeister', 'besitzer', 'manager','witwer','gatte', 'fahrer', 'banker', 'berater', 'papst', 'richter', 'behördensprecher' ])
female_words=set(['frau','kanzlerin', 'bundeskanzlerin','journalistin', 'ärztin', 'medizinerin' 'sprecherin','vorsitzende', 'politikerin', 'frauen','ihr','ihre', 'ihres', 'expertin', 'sie','partnerin', 'geliebte','mädchen','schwester','schwestern','mutter','mütter','kollegin','verlobte','ehefrau','freundin','mitschülerin','kommilitonin','großmutter','nichte','braut','enkelin','tante','vorsitzende','präsidentin','königin','dame','ihren','ihrem','priesterin', 'nonne', 'prinzessin','tochter','töchter','fußballerin','handballerin','sportlerin','athletin', 'kritikerin', 'täterin', 'ermittlerin', 'polizistin', 'anwältin', 'staatsanwältin','schülerin', 'studentin', 'geschäftsfrau', 'spielerin', 'asylbewerberin', 'bewohnerin', 'bürgermeisterin', 'besitzerin', 'managerin','witwe','gattin', 'fahrerin', 'beraterin', 'richterin','behördensprecherin'])

In [3]:
# This function defines if a sentence is male or female according to the list above
def gender_the_sentence(sentence_words):
    mw_length=len(male_words.intersection(sentence_words))
    fw_length=len(female_words.intersection(sentence_words))

    if mw_length>0 and fw_length==0:
        gender='male'
    elif mw_length==0 and fw_length>0: 
        gender='female'
    elif mw_length>0 and fw_length>0: 
        gender='both'
    else:
        gender='none'
    return gender

In [4]:
def is_it_proper(word):
        if word[0]==word[0].upper():
            case='upper'
        else:
            case='lower'
        
        word_lower=word.lower()
        try:
            proper_nouns[word_lower][case] = proper_nouns[word_lower].get(case,0)+1
        except:
            #This is triggered when the word hasn't been seen yet
            proper_nouns[word_lower]= {case:1}

In [5]:
# This function keeps count of gender frequency for each word - as in: how often was it used in m/f sentences
def increment_gender(sentence_words,gender):
    sentence_counter[gender]+=1
    word_counter[gender]+=len(sentence_words)
    for word in sentence_words:
        word_freq[gender][word]=word_freq[gender].get(word,0)+1

In [6]:
sexes=['male','female','none','both']
sentence_counter={sex:0 for sex in sexes}
word_counter={sex:0 for sex in sexes}
word_freq={sex:{} for sex in sexes}
proper_nouns={}

In [7]:
# List all input files. I had to split by month, otherwise the dataframes would crash
file_list=glob.glob('./full_months/*.csv')

In [8]:
# set stop word list of your choice here :)
import csv
with open('/home/jls/Dokumente/Lede/basics/stopwords_german.csv', 'r') as f:
    reader = csv.reader(f)
    stop_words_de = list(reader)[0]
    stop_words_de = [word.strip() for word in stop_words_de]

In [10]:
# Here's the loop of death: It reads each months' file in, split the text into sentences and words and applies the functions defined above...
# I added a "manual_list" to get rid of a handfull of words that appear in the footer of each or most articles and mess with the data analysis...
manual_list = ['heft-ausgabe', 'spiegel-magazin', '/', ':', '\\', 'spiegel', 'adblocker', 'werbung', 'freitags', 'erhältlich', 'stammt','div.thumbsmall', 'inkognito-modus', 'display:block;clear:both;overflow:hidden', 'hinzugefügt?bitte', 'do-not-track-funktionen', 'idevice', 'erweiterungen', 'margin-bottom:10px', 'adblocker', 'fußball-app', 'javascript', 'display:none', 'margin-left:px', 'text-indent', 'display:none;margin-right:px', 'fussballapp_', '.em', 'margin-right:px', 'images/image--custom-ofbc.jpg', 'div.thumbsmall{margin-right:px', 'margin-right:px', 'text-shadow', 'phone', 'kostenlose', 'klarnamen', 'border', 'url', 'color:#ffffff', 'div', 'no-repeat']
for file_name in file_list:
    df = pd.read_csv(file_name)
    df = df[~(df.actual_text.isna())]
    df['actual_text'] = df['actual_text'].str.replace('\n',' ')
    df['actual_text'] = df['actual_text'].str.replace(r'(SPIEGEL ONLINE \d+.*$)','')
    df['actual_text'] = df['actual_text'].str.replace(r'([a-z]+[A-Z]+\w+)','')
    print(file_name)
    for text in df.actual_text:
        if len(text)>500:
            #Split into sentences
            sentences=tokenizer.tokenize(text)
            for sentence in sentences:
                #word tokenize and strip punctuation
                    sentence_words=sentence.split()
                    sentence_words=[w.strip(punctuation) for w in sentence_words if len(w.strip(punctuation))>2]

                    #figure out how often each word is capitalized
                    #[is_it_proper(word) for word in sentence_words[1:]]

                    #lower case it
                    sentence_words=set([w.lower() for w in sentence_words])

                    #Figure out if there are gendered words in the sentence by computing the length of the intersection of the sets
                    gender=gender_the_sentence(sentence_words)

                    #Increment some counters
                    quality_words = [w.lower().strip() for w in sentence_words if w.lower().strip() not in stop_words_de]
                    quality_words = [w for w in quality_words if w not in manual_list]
                    increment_gender(quality_words,gender)

./full_months/full_text2015_Mai_monthly_spon.csv
./full_months/full_text2013_November_monthly_spon.csv
./full_months/full_text2017_September_monthly_spon.csv
./full_months/full_text2017_Juli_monthly_spon.csv
./full_months/full_text2016_Juli_monthly_spon.csv
./full_months/full_text2016_August_monthly_spon.csv
./full_months/full_text2013_April_monthly_spon.csv
./full_months/full_text2015_Januar_monthly_spon.csv
./full_months/full_text2012_Oktober_monthly_spon.csv
./full_months/full_text2013_Dezember_monthly_spon.csv
./full_months/full_text2016_März_monthly_spon.csv
./full_months/full_text2015_März_monthly_spon.csv
./full_months/full_text2012_Februar_monthly_spon.csv
./full_months/full_text2014_Januar_monthly_spon.csv
./full_months/full_text2013_Oktober_monthly_spon.csv
./full_months/full_text2012_September_monthly_spon.csv
./full_months/full_text2014_Juni_monthly_spon.csv
./full_months/full_text2017_Mai_monthly_spon.csv
./full_months/full_text2017_April_monthly_spon.csv
./full_months/ful

In [12]:
#Here's a list of the most common words vor females
common_words=set(w for w in [w for w in sorted (word_freq['female'],key=word_freq['female'].get,reverse=True)[:1000]]+[w for w in sorted (word_freq['male'],key=word_freq['male'].get,reverse=True)[:1000]] if w not in stop_words_de)
common_words=list(common_words-male_words-female_words)
print(common_words[:50])

['nahmen', 'wladimir', 'gestiegen', 'rennen', 'richtig', '2015', 'ressortleiter', 'spd', 'beachten', 'grenze', 'worten', 'wohnung', 'kleine', 'meister', 'spitze', 'thema', 'bekam', 'offenbar', 'wusste', 'studium', 'passiert', 'schrieb', 'ansicht', 'vergangenen', 'olympischen', 'tod', 'verhandlungen', 'aktiviert', 'donnerstag', 'bekommen', 'aktuellen', 'rüber', 'betroffenen', 'trump', 'deutsch', 'angreifer', 'entscheiden', 'verantwortung', 'schließen', 'läuft', 'montag', 'sorge', 'britische', 'stolz', 'vergleich', 'konflikt', 'gingen', 'fest', 'botschaft', 'bilder']


In [13]:
# Here's the calculation for the relative frequency of each word
male_percent={word:(word_freq['male'].get(word,0) / word_counter['male']) 
              / (word_freq['female'].get(word,0) / word_counter['female']+word_freq['male'].get(word,0)/word_counter['male']) for word in common_words}

In [14]:
# Let's look at some basic stats!
print('%.1f%% gendered' % (100*(sentence_counter['male']+sentence_counter['female'])/(sentence_counter['male']+sentence_counter['female']+sentence_counter['both']+sentence_counter['none'])))
print('%s sentences about men.' % sentence_counter['male'])
print('%s sentences about women.' % sentence_counter['female'])
print('%.1f sentences about men for each sentence about women.' % (sentence_counter['male']/sentence_counter['female']))

45.8% gendered
2886119 sentences about men.
719419 sentences about women.
4.0 sentences about men for each sentence about women.


In [15]:
# More stats: Top male words and top female words. This takes the relative frequency into account and also gives a ratio.
# Unfortunately, we see false positives here: 'von der Leyen' is the name of the German (female) minister of defence, but 'der' is also a male article.
# As 'Leyen' only appears in the phrase 'von der Leyen' it is calculated as 100 per cent male by the algorithm.

header ='Ratio\tMale\tFemale\tWord'
print('Male words')
print(header)
for word in sorted (male_percent,key=male_percent.get,reverse=True)[:250]:
    try:
        ratio=male_percent[word]/(1-male_percent[word])
    except:
        ratio=100
    print('%.1f\t%02d\t%02d\t%s' % (ratio,word_freq['male'].get(word,0),word_freq['female'].get(word,0),word))

print('\n\n')
print('Female words')
print(header)
for word in sorted (male_percent,key=male_percent.get,reverse=False)[:250]:
    try:
        ratio=(1-male_percent[word])/male_percent[word]
    except:
        ratio=100
    print('%.1f\t%01d\t%01d\t%s' % (ratio,word_freq['male'].get(word,0),word_freq['female'].get(word,0),word))

Male words
Ratio	Male	Female	Word
100.0	3536	00	leyen
198.8	3369	03	ressortleiter
100.9	4558	08	korrespondent
47.7	8622	32	journalistenschule
41.1	8591	37	redakteur
13.3	6550	87	university
11.0	4605	74	provinz
10.5	3668	62	rundfunk
10.3	4546	78	nzz
9.8	3637	66	mitarbeit
9.6	10278	190	league
9.1	4642	90	volontariat
9.0	4651	91	bvb
8.9	4173	83	erzielte
8.5	3511	73	halbzeit
8.3	3347	71	axel
8.2	10185	219	konzern
7.9	5344	120	bundesliga
7.9	5634	127	leiter
7.6	11343	263	universität
7.4	7240	173	liga
7.3	3847	93	küste
7.3	5801	141	champions
7.0	7792	197	faz
7.0	3580	91	hsv
6.5	3965	108	politikwissenschaft
6.5	4639	127	nationalspieler
6.2	3352	95	vfl
6.2	15779	451	saison
6.2	5072	145	klub
5.9	9382	280	taz
5.7	3899	122	stürmer
5.6	4780	152	tageszeitung
5.6	4088	130	ddr
5.4	11457	373	hauptstadt
5.2	3716	126	kapitän
5.2	5071	174	regel
5.1	9466	326	region
5.1	4505	157	journalist
5.0	9290	327	nachrichtenagentur
5.0	6398	226	1999
5.0	5054	180	treffer
4.9	6848	245	schweiz
4.8	10350	378	ukraine
4.8	

In [19]:
# Save to file
outfile_name='gender.tsv'
tsv_outfile=open(outfile_name,'w')
header='percent_male\tmale_count\tfemalecount\tword\n'
tsv_outfile.write(header)
for word in common_words:
    row = '%.2f\t%01d\t%01d\t%s\n' % (100*male_percent[word],word_freq['male'].get(word,0),word_freq['female'].get(word,0),word)
    tsv_outfile.write(row)
tsv_outfile.close()

In [17]:
# relative frequency for the most common words overall
all_words=[w for w in word_freq['none']]+[w for w in word_freq['both']]+[w for w in word_freq['male']]+[w for w in word_freq['female']]
all_words={w:(word_freq['male'].get(w,0)+word_freq['female'].get(w,0)+word_freq['both'].get(w,0)+word_freq['none'].get(w,0)) for w in set(all_words)}

print('word\t\tMale\tFemale')
for word in sorted (all_words,key=all_words.get,reverse=True)[:100]:
    print('%s\t\t%.1f%%\t%.1f%%' % (word,100*word_freq['male'].get(word,0)/sentence_counter['male'],100*word_freq['female'].get(word,0)/sentence_counter['female']))

word		Male	Female
menschen		1.6%	1.9%
jahren		1.9%	1.5%
jahr		1.7%	1.1%
deutschland		1.8%	1.1%
euro		1.6%	0.9%
prozent		1.8%	0.9%
deutschen		2.0%	0.7%
jahre		1.6%	1.1%
zeit		1.3%	1.0%
berlin		1.2%	0.7%
millionen		1.1%	0.6%
welt		1.5%	0.5%
usa		1.1%	0.7%
vergangenen		1.3%	0.6%
leben		0.9%	1.3%
deutsche		1.1%	0.6%
hamburg		1.1%	0.4%
land		0.9%	0.7%
regierung		1.0%	0.6%
polizei		0.9%	0.4%
große		0.8%	0.5%
lange		0.6%	0.6%
großen		0.8%	0.5%
geld		0.6%	0.8%
frauen		0.0%	3.2%
offenbar		0.7%	0.5%
woche		0.8%	0.3%
unternehmen		0.6%	0.5%
heißt		0.7%	0.4%
uhr		0.6%	0.4%
angaben		0.8%	0.3%
the		0.5%	0.4%
tag		0.6%	0.6%
zuvor		0.6%	0.4%
new		0.7%	0.3%
geschichte		0.8%	0.4%
frage		0.6%	0.5%
kurz		0.7%	0.4%
stadt		0.8%	0.2%
2014		0.5%	0.2%
milliarden		0.6%	0.3%
kinder		0.4%	1.1%
zeitung		0.8%	0.2%
2011		0.5%	0.2%
merkel		0.3%	1.3%
deutlich		0.6%	0.4%
spiel		0.6%	0.3%
europa		0.6%	0.4%
wochen		0.6%	0.4%
lässt		0.6%	0.4%
platz		0.6%	0.3%
teil		0.7%	0.4%
2015		0.5%	0.3%
stehen		0.5%	0.5%
münchen		0.7%	